In [ ]:
import os
import numpy as np
import pandas as pd
from PIL import Image
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D, BatchNormalization
from tensorflow.keras.applications import ResNet101  # Import ResNet101
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf

# Parameters
dataset_path = r"E:\Niyas\dataset"  # Replace with your dataset path
metadata_file = os.path.join(dataset_path, "edvard_munch.csv")
image_dir = os.path.join(dataset_path, "munch_paintings")
image_size = (224, 224)  # Larger input size
batch_size = 16
initial_epochs = 50
fine_tune_epochs = 20
initial_learning_rate = 1e-3
fine_tune_learning_rate = 1e-5

# Load metadata
if os.path.exists(metadata_file):
    metadata = pd.read_csv(metadata_file)
    print("Metadata loaded successfully!")
else:
    raise FileNotFoundError("Metadata file not found!")

# Filter and clean metadata
required_columns = ["name", "year", "location", "status", "technique", "filename"]
metadata = metadata[required_columns].dropna(subset=["filename"])
metadata["filename"] = metadata["filename"].astype(str).str.strip()
metadata["image_path"] = metadata["filename"].apply(lambda x: os.path.join(image_dir, x))
metadata = metadata[metadata["image_path"].apply(os.path.exists)]

if metadata.empty:
    raise ValueError("No valid images found in metadata!")

# Load and preprocess images
def load_and_preprocess_image(image_path):
    try:
        img = Image.open(image_path).convert("RGB").resize(image_size)
        img_array = np.array(img, dtype=np.float32)
        return tf.keras.applications.resnet.preprocess_input(img_array)  # Preprocess for ResNet
    except Exception as e:
        print(f"Error loading image {image_path}: {e}")
        return None

images = []
labels = []
for _, row in metadata.iterrows():
    image = load_and_preprocess_image(row["image_path"])
    if image is not None:
        images.append(image)
        labels.append(row["name"])  # Use 'name' column as the label

images = np.array(images)
labels = np.array(labels)

# Encode labels
label_encoder = LabelEncoder()
labels_encoded = label_encoder.fit_transform(labels)

# One-hot encode labels
num_classes = len(label_encoder.classes_)
print(f"Number of unique classes: {num_classes}")
y_one_hot = to_categorical(labels_encoded, num_classes=num_classes)

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(images, y_one_hot, test_size=0.2, random_state=42)

# Define data augmentation
train_data_gen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode="nearest"
)

val_data_gen = ImageDataGenerator()  # Validation data will not be augmented

train_gen = train_data_gen.flow(X_train, y_train, batch_size=batch_size)
val_gen = val_data_gen.flow(X_test, y_test, batch_size=batch_size)

# Define model using ResNet50
base_model = ResNet101(weights="imagenet", include_top=False, input_shape=(224, 224, 3))  # Use ResNet101

model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dense(256, activation="relu", kernel_regularizer=tf.keras.regularizers.l2(0.01)),
    BatchNormalization(),
    Dropout(0.5),
    Dense(num_classes, activation="softmax")
])

# Freeze base model layers for initial training
base_model.trainable = False

# Compile the model
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=initial_learning_rate),
    loss="categorical_crossentropy",
    metrics=["accuracy"]
)

# Compute class weights
from sklearn.utils.class_weight import compute_class_weight
class_weights = compute_class_weight(
    class_weight='balanced',
    classes=np.unique(labels_encoded),
    y=labels_encoded
)
class_weights = dict(enumerate(class_weights))

# Train the model
history = model.fit(
    train_gen,
    steps_per_epoch=len(X_train) // batch_size,
    validation_data=val_gen,
    validation_steps=len(X_test) // batch_size,
    epochs=initial_epochs,
    class_weight=class_weights  # Apply class weights
)

# Fine-tuning: Unfreeze some layers of the base model
base_model.trainable = True
for layer in base_model.layers[:-30]:  # Freeze earlier layers
    layer.trainable = False

# Compile with a smaller learning rate
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=fine_tune_learning_rate),
    loss="categorical_crossentropy",
    metrics=["accuracy"]
)

# Fine-tune the model
history_fine = model.fit(
    train_gen,
    steps_per_epoch=len(X_train) // batch_size,
    validation_data=val_gen,
    validation_steps=len(X_test) // batch_size,
    epochs=fine_tune_epochs,
    class_weight=class_weights
)

# Save the model and label encoder
model.save("painting_recognition_model_resnet50_finetuned_1.h5")
np.save("label_classes_resnet50_1.npy", label_encoder.classes_)
print("Model and label encoder saved!")

# Prediction Function
def predict_image(image_path):
    image = load_and_preprocess_image(image_path)
    if image is None:
        print("Invalid image!")
        return
    image = np.expand_dims(image, axis=0)  # Add batch dimension
    predictions = model.predict(image)
    predicted_class_idx = np.argmax(predictions)
    predicted_class = label_encoder.inverse_transform([predicted_class_idx])[0]
    print(f"Predicted Class: {predicted_class}")
    print("Image Information:")
    print(metadata[metadata["name"] == predicted_class])

# Test prediction
test_image_path = r"E:/Niyas/dataset/munch_paintings/86.jpg"  # Replace with the path to a test image
predict_image(test_image_path)


#  Define model using ResNet101





Metadata loaded successfully!
Number of unique classes: 1211
Epoch 1/50
88/88 [==============================] - 35s 87ms/step - loss: 10.7455 - accuracy: 0.0014 - val_loss: 9.6785 - val_accuracy: 0.0028
Epoch 2/50
88/88 [==============================] - 7s 80ms/step - loss: 8.8189 - accuracy: 0.0229 - val_loss: 8.7800 - val_accuracy: 0.0085
Epoch 3/50
88/88 [==============================] - 7s 81ms/step - loss: 7.8066 - accuracy: 0.0793 - val_loss: 8.2783 - val_accuracy: 0.0341
Epoch 4/50
88/88 [==============================] - 7s 82ms/step - loss: 7.0669 - accuracy: 0.1222 - val_loss: 8.0818 - val_accuracy: 0.0227
Epoch 5/50
88/88 [==============================] - 7s 83ms/step - loss: 6.4884 - accuracy: 0.1751 - val_loss: 7.9690 - val_accuracy: 0.0341
Epoch 6/50
88/88 [==============================] - 7s 83ms/step - loss: 5.9062 - accuracy: 0.2330 - val_loss: 8.1787 - val_accuracy: 0.0398
Epoch 7/50
88/88 [==============================] - 7s 83ms/step - loss: 5.3315 - accuracy:

In [7]:
from tensorflow.keras.models import load_model
import numpy as np
import tensorflow as tf
import os
from PIL import Image
from sklearn.preprocessing import LabelEncoder
import pandas as pd
from dotenv import load_dotenv

load_dotenv()

# Retrieve the API key
api_key = os.getenv("API_KEY")

model = load_model("painting_recognition_model_resnet50_finetuned_1.h5")
label_classes = np.load("label_classes_resnet50_1.npy", allow_pickle=True)
label_encoder = LabelEncoder()
label_encoder.classes_ = label_classes
image_size = (224, 224)
dataset_path = r"E:\Niyas\dataset"  # Replace with your dataset path
metadata_file = os.path.join(dataset_path, "edvard_munch.csv")
image_dir = os.path.join(dataset_path, "munch_paintings")

if os.path.exists(metadata_file):
    metadata = pd.read_csv(metadata_file)
    print("Metadata loaded successfully!")
else:
    raise FileNotFoundError("Metadata file not found!")

# Filter and clean metadata
required_columns = ["name", "year", "location", "status", "technique", "filename"]
metadata = metadata[required_columns].dropna(subset=["filename"])
metadata["filename"] = metadata["filename"].astype(str).str.strip()
metadata["image_path"] = metadata["filename"].apply(lambda x: os.path.join(image_dir, x))
metadata = metadata[metadata["image_path"].apply(os.path.exists)]

if metadata.empty:
    raise ValueError("No valid images found in metadata!")


def load_and_preprocess_image(image_path):
    try:
        img = Image.open(image_path).convert("RGB").resize(image_size)
        img_array = np.array(img, dtype=np.float32)
        return tf.keras.applications.resnet.preprocess_input(img_array)  # Preprocess for ResNet
    except Exception as e:
        print(f"Error loading image {image_path}: {e}")
        return None


def predict_image(image_path):
    image = load_and_preprocess_image(image_path)
    if image is None:
        print("Invalid image!")
        return
    image = np.expand_dims(image, axis=0)  # Add batch dimension
    predictions = model.predict(image)
    predicted_class_idx = np.argmax(predictions)
    predicted_class = label_encoder.inverse_transform([predicted_class_idx])[0]
    print(f"Predicted Class: {predicted_class}")
    print("Image Information:")
    print(metadata[metadata["name"] == predicted_class])
    
    search_query = predicted_class+" by Edvard Munch"
    print(search_query)
    results = fetch_google_search_results(search_query,api_key, cx='20b79b15ec0d844c2')
    for result in results:
        print(f"Title: {result['title']}")
        print(f"URL: {result['link']}")
        print(f"Snippet: {result['snippet']}\n")
    # result = fetch_wikipedia_summary(predicted_class)
    # if result:
    #     print(f"Title: {result['title']}")
    #     print(f"Description: {result['description']}")
    #     print(f"URL: {result['url']}")

import requests
from urllib.parse import quote

# def fetch_wikipedia_summary(topic):
#     """
#     Fetch a summary of the given topic from Wikipedia.
    
#     Args:
#         topic (str): The topic to search for on Wikipedia.
    
#     Returns:
#         dict: A dictionary with the title, description, and URL of the page.
#     """
#     try:
#         # Format the topic for the API (replace spaces with underscores)
#         formatted_topic = topic.replace(" ", "_")
#         url = f"https://en.wikipedia.org/api/rest_v1/page/summary/{formatted_topic}"
        
#         # Make the request
#         response = requests.get(url)
#         response.raise_for_status()  # Raise an HTTPError for bad responses (4xx, 5xx)
        
#         # Parse the JSON response
#         data = response.json()
#         return {
#             "title": data.get("title", "N/A"),
#             "description": data.get("extract", "No description available."),
#             "url": data.get("content_urls", {}).get("desktop", {}).get("page", "N/A")
#         }
#     except requests.exceptions.RequestException as e:
#         print(f"Error fetching Wikipedia summary: {e}")
#         return None
import requests
from urllib.parse import quote

def fetch_google_search_results(query, api_key, cx):
    """
    Fetches search results from Google Custom Search API.
    
    Args:
        query (str): The search query to search on Google.
        api_key (str): Your Google API key.
        cx (str): Your Custom Search Engine ID (CX).
    
    Returns:
        list: A list of dictionaries containing search result information.
    """
    try:
        # URL encode the query
        encoded_query = quote(query)
        
        # Construct the API request URL
        search_url = f"https://www.googleapis.com/customsearch/v1?q={encoded_query}&key={api_key}&cx={cx}"
        
        # Make the request
        response = requests.get(search_url)
        response.raise_for_status()  # Raise an HTTPError for bad responses (4xx, 5xx)
        
        # Parse the JSON response
        search_data = response.json()
        
        # Extract relevant information from search results
        search_results = []
        for item in search_data.get("items", []):
            result = {
                "title": item["title"],
                "link": item["link"],
                "snippet": item["snippet"]
            }
            search_results.append(result)
        
        return search_results
    
    except requests.exceptions.RequestException as e:
        print(f"Error fetching search results: {e}")
        return []

# Usage example
# api_key = 'YOUR_GOOGLE_API_KEY'  # Replace with your Google API key
# cx = 'YOUR_CUSTOM_SEARCH_ENGINE_ID'  # Replace with your Custom Search Engine ID

# Query for the artwork by Edvard Munch



# Print search results



    

# Test prediction
test_image_path = r"E:/Niyas/dataset/munch_paintings/1.jpg"  # Replace with the path to a test image
predict_image(test_image_path)

Metadata loaded successfully!
1/1 [==============================] - 1s 1s/step
Predicted Class: Telthusbakken with Gamle Aker Church
Image Information:
                                   name  year          location status  \
0  Telthusbakken with Gamle Aker Church  1880  Location unknown    NaN   

      technique filename                              image_path  
0   carton, oil    1.jpg  E:\Niyas\dataset\munch_paintings\1.jpg  
Telthusbakken with Gamle Aker Church by Edvard Munch
Title: File:Edvard Munch - Telthusbakken with Gamle Aker Church (1880 ...
URL: https://commons.wikimedia.org/wiki/File:Edvard_Munch_-_Telthusbakken_with_Gamle_Aker_Church_(1880).jpg
Snippet: Jun 12, 2024 ... This is a faithful photographic reproduction of a two-dimensional, public domain work of art. The work of art itself is in the public domain.

Title: Edvard Munch Telthusbakken with Gamle Aker ... - Amazon.com
URL: https://www.amazon.com/Telthusbakken-Painting-Picture-Unframed-12x18inch/dp/B0C3MWYY11
S

In [6]:
! pip install python-dotenv

  Using cached python_dotenv-1.0.1-py3-none-any.whl.metadata (23 kB)
Using cached python_dotenv-1.0.1-py3-none-any.whl (19 kB)
